In [ ]:
import cv2

# 이미지 불러오기
img = cv2.imread("/home/seyeon/hakathon/Soomgil/backend/app/services/path_image/rabbit.png")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 블러 + 캐니 엣지 추출
blurred = cv2.GaussianBlur(gray, (5,5), 0)
edges = cv2.Canny(blurred, 100, 200)

# 윤곽선 찾기
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 원본에 윤곽선 그리기
img_contour = img.copy()
cv2.drawContours(img_contour, contours, -1, (0,255,0), 2)

# 화면에 표시
cv2.imshow("Edges", edges)
cv2.imshow("Contours", img_contour)
cv2.waitKey(0)
cv2.destroyAllWindows()




In [13]:
import cv2
import numpy as np
import pandas as pd
import geopandas as gpd
import networkx as nx
import folium
from sklearn.neighbors import BallTree
from geopy.distance import geodesic
from scipy.spatial.distance import directed_hausdorff
from IPython.display import IFrame

# ----------------
# 1. contour 추출
# ----------------
def contour_length(contour):
    return np.sum(np.sqrt(np.sum(np.diff(contour, axis=0)**2, axis=1)))

img = cv2.imread("/home/seyeon/hakathon/Soomgil/backend/app/services/path_image/heart.png")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

_, binary = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)
kernel = np.ones((5,5), np.uint8)
closed = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contour = max(contours, key=cv2.contourArea)
epsilon = 3.0
approx = cv2.approxPolyDP(contour, epsilon, True)
contour = approx.reshape(-1, 2)

# contour 픽셀 단위 둘레
orig_len = contour_length(contour)

# ----------------
# 2. 목표 산책거리 (예: 7km)
# ----------------
target_len = 5000  # 7 km

# ----------------
# 3. 네트워크 로딩
# ----------------
nodes = pd.read_csv("/home/seyeon/hakathon/Soomgil/data/04_final_data/final_nodes.csv")
edges = gpd.read_file("/home/seyeon/hakathon/Soomgil/data/04_final_data/final_edges.geojson")

G = nx.Graph()
for _, row in nodes.iterrows():
    G.add_node(row["osmid"], x=row["lon"], y=row["lat"])
for _, row in edges.iterrows():
    G.add_edge(row["u"], row["v"], weight=row["length"])

# bbox
min_lon, max_lon = nodes["lon"].min(), nodes["lon"].max()
min_lat, max_lat = nodes["lat"].min(), nodes["lat"].max()
center_lat, center_lon = nodes["lat"].mean(), nodes["lon"].mean()

# bbox 실제 크기(m) 계산
bbox_width_m = geodesic((center_lat, min_lon), (center_lat, max_lon)).m
bbox_height_m = geodesic((min_lat, center_lon), (max_lat, center_lon)).m

# contour bbox (픽셀 단위)
contour_w = contour[:,0].max() - contour[:,0].min()
contour_h = contour[:,1].max() - contour[:,1].min()

# 픽셀 → 미터 변환 계수 (비율 고정: 단일 값)
px_to_m_x = bbox_width_m / contour_w
px_to_m_y = bbox_height_m / contour_h
px_to_m = (px_to_m_x + px_to_m_y) / 2

# ----------------
# 4. contour 크기 스케일 (원본 비율 고정)
# ----------------
orig_len_m = orig_len * px_to_m
scale_factor = target_len / orig_len_m
contour_scaled = (contour - contour.mean(0)) * scale_factor + contour.mean(0)

# ----------------
# 5. contour 픽셀 → 위경도 변환 (비율 고정)
# ----------------
def contour_to_geo(c, dx=0, dy=0):
    geo_points = []
    for px, py in c:
        lon = center_lon + (px - contour_scaled[:,0].mean()) * (px_to_m / 111320) + dx
        lat = center_lat - (py - contour_scaled[:,1].mean()) * (px_to_m / 110540) + dy
        geo_points.append((lat, lon))
    return geo_points

# ----------------
# 6. snap + 실제 경로 생성 함수
# ----------------
coords = np.vstack((nodes["lat"], nodes["lon"])).T
tree = BallTree(np.radians(coords), metric="haversine")

def build_route(contour_geo):
    contour_sampled = contour_geo[::max(1, len(contour_geo)//100)]
    
    snapped_nodes = []
    for lat, lon in contour_sampled:
        _, idx = tree.query([np.radians([lat, lon])], k=1)
        node = nodes.iloc[idx[0][0]]
        snapped_nodes.append(node["osmid"])
    
    route_coords = []
    for i in range(len(snapped_nodes) - 1):
        u, v = snapped_nodes[i], snapped_nodes[i+1]
        try:
            path = nx.shortest_path(G, u, v, weight="weight")
            for n in path:
                route_coords.append((G.nodes[n]["y"], G.nodes[n]["x"]))
        except nx.NetworkXNoPath:
            continue
    return route_coords

# ----------------
# 7. 유사도 측정 (Hausdorff distance)
# ----------------
def hausdorff_distance(A, B):
    if len(A)==0 or len(B)==0:
        return float("inf")
    return max(directed_hausdorff(A, B)[0], directed_hausdorff(B, A)[0])

# ----------------
# 8. 슬라이딩 탐색
# ----------------
best_path = None
best_score = float("inf")
best_contour = None

# 슬라이딩 (위치 이동 범위 및 간격 조정 가능)
dx_vals = np.linspace(-0.01, 0.01, 5)  # 약 ±1km 정도
dy_vals = np.linspace(-0.01, 0.01, 5)

for dx in dx_vals:
    for dy in dy_vals:
        shifted = contour_to_geo(contour_scaled, dx=dx, dy=dy)
        route_coords = build_route(shifted)
        if not route_coords:
            continue
        score = hausdorff_distance(np.array(shifted), np.array(route_coords))
        if score < best_score:
            best_score = score
            best_path = route_coords
            best_contour = shifted

print("Best Hausdorff distance:", best_score)

# ----------------
# 9. 시각화
# ----------------
m = folium.Map(location=[center_lat, center_lon], zoom_start=14, tiles="cartodbpositron")

# 네트워크 전체 (회색)
for _, row in edges.iterrows():
    coords_edge = list(row["geometry"].coords)
    folium.PolyLine([(lat, lon) for lon, lat in coords_edge],
                    color="gray", weight=1, opacity=0.3).add_to(m)

# 빨강 윤곽선 (최적 위치)
if best_contour:
    folium.PolyLine(best_contour, color="red", weight=2, opacity=0.9, tooltip="최적 윤곽선").add_to(m)

# 파랑 경로 (최적 산책 경로)
if best_path:
    folium.PolyLine(best_path, color="blue", weight=3, opacity=0.9, tooltip="산책 경로").add_to(m)

m.save("map.html")
IFrame("map.html", width=900, height=600)


Skipping field name: unsupported OGR type: 5


Best Hausdorff distance: 0.002040615373277473


In [ ]:
import cv2
import numpy as np
import pandas as pd
import geopandas as gpd
import networkx as nx
import folium
from sklearn.neighbors import BallTree
from geopy.distance import geodesic
from IPython.display import IFrame
from scipy.spatial.distance import euclidean
from functools import lru_cache

# ----------------
# 1. contour 추출
# ----------------
def contour_length(contour):
    return np.sum(np.sqrt(np.sum(np.diff(contour, axis=0)**2, axis=1)))

img = cv2.imread("/home/seyeon/hakathon/Soomgil/backend/app/services/path_image/jellyfish.png")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

_, binary = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)
kernel = np.ones((5,5), np.uint8)
closed = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contour = max(contours, key=cv2.contourArea)
epsilon = 3.0
approx = cv2.approxPolyDP(contour, epsilon, True)
contour = approx.reshape(-1, 2)

# contour 픽셀 단위 둘레
orig_len = contour_length(contour)

# ----------------
# 2. 목표 산책거리 (예: 7km)
# ----------------
target_len = 5000  # 7 km

# ----------------
# 3. 네트워크 로딩
# ----------------
nodes = pd.read_csv("/home/seyeon/hakathon/Soomgil/data/04_final_data/final_nodes.csv")
edges = gpd.read_file("/home/seyeon/hakathon/Soomgil/data/04_final_data/final_edges.geojson")

G = nx.Graph()
for _, row in nodes.iterrows():
    G.add_node(row["osmid"], x=row["lon"], y=row["lat"])
for _, row in edges.iterrows():
    G.add_edge(row["u"], row["v"], weight=row["length"])

# bbox
min_lon, max_lon = nodes["lon"].min(), nodes["lon"].max()
min_lat, max_lat = nodes["lat"].min(), nodes["lat"].max()
center_lat, center_lon = nodes["lat"].mean(), nodes["lon"].mean()

# bbox 실제 크기(m) 계산
bbox_width_m = geodesic((center_lat, min_lon), (center_lat, max_lon)).m
bbox_height_m = geodesic((min_lat, center_lon), (max_lat, center_lon)).m

# contour bbox (픽셀 단위)
contour_w = contour[:,0].max() - contour[:,0].min()
contour_h = contour[:,1].max() - contour[:,1].min()

# 픽셀 → 미터 변환 계수 (비율 고정: 단일 값)
px_to_m_x = bbox_width_m / contour_w
px_to_m_y = bbox_height_m / contour_h
px_to_m = (px_to_m_x + px_to_m_y) / 2

# ----------------
# 4. contour 크기 스케일 (사진 가로세로 비율 고정)
# ----------------

# contour 원본 폭/높이 (픽셀 단위)
contour_w = contour[:,0].max() - contour[:,0].min()
contour_h = contour[:,1].max() - contour[:,1].min()

# contour의 둘레길이를 "픽셀 단위"로 계산
orig_len_px = contour_length(contour)

# 실제 길이로 변환할 scale factor (단일 값)
scale_factor = target_len / (orig_len_px * px_to_m)

# x, y축 동일 scale 적용 → 원본 비율 유지
contour_scaled = (contour - contour.mean(0)) * scale_factor + contour.mean(0)

# ----------------
# 5. contour 픽셀 → 위경도 변환 (비율 고정)
# ----------------
def contour_to_geo(c, dx=0, dy=0):
    geo_points = []
    for px, py in c:
        # 픽셀 → 미터 (비율 고정: 동일 스케일 사용)
        dx_m = (px - contour_scaled[:,0].mean()) * px_to_m
        dy_m = (py - contour_scaled[:,1].mean()) * px_to_m

        # 미터 → 위경도 변환 (위도/경도 계수 따로 적용)
        lon = center_lon + (dx_m / (111320 * np.cos(np.radians(center_lat)))) + dx
        lat = center_lat - (dy_m / 110540) + dy

        geo_points.append((lat, lon))
    return geo_points

# ----------------
# 6. edge 방향 기반 비용 함수
# ----------------
def edge_cost(u, n, target_vec, G):
    ux, uy = G.nodes[u]["x"], G.nodes[u]["y"]
    nx_, ny_ = G.nodes[n]["x"], G.nodes[n]["y"]

    vec = np.array([nx_ - ux, ny_ - uy])
    if np.linalg.norm(vec) < 1e-6:
        return 1e9

    cos_sim = np.dot(vec, target_vec) / (np.linalg.norm(vec) * np.linalg.norm(target_vec) + 1e-9)
    dist = np.linalg.norm(vec)

    return dist * (1 - cos_sim)

# ----------------
# 7. snap + 모양 보존 경로 생성
# ----------------
coords = np.vstack((nodes["lat"], nodes["lon"])).T
tree = BallTree(np.radians(coords), metric="haversine")

def build_route(contour_geo):
    contour_sampled = contour_geo[::max(1, len(contour_geo)//100)]

    snapped_nodes = []
    for lat, lon in contour_sampled:
        _, idx = tree.query([np.radians([lat, lon])], k=1)
        node = nodes.iloc[idx[0][0]]
        snapped_nodes.append(node["osmid"])

    route_coords = []
    for i in range(len(snapped_nodes) - 1):
        u, v = snapped_nodes[i], snapped_nodes[i+1]

        ux, uy = G.nodes[u]["x"], G.nodes[u]["y"]
        vx, vy = G.nodes[v]["x"], G.nodes[v]["y"]
        target_vec = np.array([vx - ux, vy - uy])

        def weight_func(a, b, d):
            return edge_cost(a, b, target_vec, G)

        try:
            path = nx.dijkstra_path(G, u, v, weight=weight_func)
            for n in path:
                route_coords.append((G.nodes[n]["y"], G.nodes[n]["x"]))
        except nx.NetworkXNoPath:
            continue
    return route_coords

# ----------------
# 8. Fréchet distance 계산
# ----------------
@lru_cache(maxsize=None)
def _c(i, j, P, Q):
    if i == 0 and j == 0:
        return euclidean(P[0], Q[0])
    elif i > 0 and j == 0:
        return max(_c(i-1, 0, P, Q), euclidean(P[i], Q[0]))
    elif i == 0 and j > 0:
        return max(_c(0, j-1, P, Q), euclidean(P[0], Q[j]))
    elif i > 0 and j > 0:
        return max(
            min(_c(i-1, j, P, Q), _c(i-1, j-1, P, Q), _c(i, j-1, P, Q)),
            euclidean(P[i], Q[j])
        )
    else:
        return float("inf")

def frechet_distance(P, Q):
    P = tuple(map(tuple, P))
    Q = tuple(map(tuple, Q))
    return _c(len(P)-1, len(Q)-1, P, Q)

# ----------------
# 9. 슬라이딩 탐색 (동대문구 전체 범위)
# ----------------
best_path = None
best_score = float("inf")
best_contour = None

# bbox 안에서 일정 간격으로 슬라이딩
dx_vals = np.linspace(min_lon - center_lon, max_lon - center_lon, 15)  # 경도 방향
dy_vals = np.linspace(min_lat - center_lat, max_lat - center_lat, 15)  # 위도 방향

for dx in dx_vals:
    for dy in dy_vals:
        shifted = contour_to_geo(contour_scaled, dx=dx, dy=dy)
        route_coords = build_route(shifted)
        if not route_coords:
            continue
        score = frechet_distance(np.array(shifted), np.array(route_coords))
        if score < best_score:
            best_score = score
            best_path = route_coords
            best_contour = shifted

print("Best Fréchet distance:", best_score)


# ----------------
# 10. 시각화
# ----------------
m = folium.Map(location=[center_lat, center_lon], zoom_start=14, tiles="cartodbpositron")

# 네트워크 전체 (회색)
for _, row in edges.iterrows():
    coords_edge = list(row["geometry"].coords)
    folium.PolyLine([(lat, lon) for lon, lat in coords_edge],
                    color="gray", weight=1, opacity=0.3).add_to(m)

# 빨강 윤곽선 (최적 위치)
if best_contour:
    folium.PolyLine(best_contour, color="red", weight=2, opacity=0.9, tooltip="최적 윤곽선").add_to(m)

# 파랑 경로 (엣지 기반 + Fréchet distance 최적화)
if best_path:
    folium.PolyLine(best_path, color="blue", weight=3, opacity=0.9, tooltip="산책 경로").add_to(m)

m.save("map.html")
IFrame("map.html", width=900, height=600)


[ WARN:0@9118.491] global loadsave.cpp:275 findDecoder imread_('/home/seyeon/hakathon/Soomgil/backend/app/services/path_image/jellyfish.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.12.0) /io/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
